# Build an Agent
## End-to-end agent

In [ ]:
# Import relevant functionality
import os
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = init_chat_model("anthropic:claude-3-5-sonnet-latest")
search = TavilySearch(tavily_api_key="TAVILY_API_KEY", max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [131]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Hi, I'm Bob and I live in SF.",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob and I live in SF.


TypeError: "Could not resolve authentication method. Expected either api_key or auth_token to be set. Or for one of the `X-Api-Key` or `Authorization` headers to be explicitly omitted"

In [132]:
input_message = {
    "role": "user",
    "content": "What's the weather where I live?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's the weather where I live?


TypeError: "Could not resolve authentication method. Expected either api_key or auth_token to be set. Or for one of the `X-Api-Key` or `Authorization` headers to be explicitly omitted"

## Setup
## Jupyter Notebook

In [133]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [ ]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [134]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'error': ValueError('Error 401: Unauthorized: missing or invalid API key.')}


## Using Language Models

In [135]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [136]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 API key expired. Please renew the API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key expired. Please renew the API key."
]

In [137]:
model_with_tools = model.bind_tools(tools)

In [138]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 API key expired. Please renew the API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key expired. Please renew the API key."
]

In [139]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 API key expired. Please renew the API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key expired. Please renew the API key."
]

## Create the agent

In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent

In [ ]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 API key expired. Please renew the API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key expired. Please renew the API key."
]

In [ ]:
input_message = {"role": "user", "content": "Search for the weather in SF"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 API key expired. Please renew the API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key expired. Please renew the API key."
]

## Streaming Messages

In [ ]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in SF


ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 API key expired. Please renew the API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key expired. Please renew the API key."
]

## Streaming tokens

In [ ]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 API key expired. Please renew the API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key expired. Please renew the API key."
]

## Adding in memory

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
for step in agent_executor.stream(
    {"messages": [("user", "Hi, I'm Bob!")]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob!


ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 API key expired. Please renew the API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key expired. Please renew the API key."
]

In [ ]:
for step in agent_executor.stream(
    {"messages": [("user", "What is my name?")]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is my name?


ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 API key expired. Please renew the API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key expired. Please renew the API key."
]

In [ ]:
config = {"configurable": {"thread_id": "xyz123"}}

for step in agent_executor.stream(
    {"messages": [("user", "What is my name?")]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is my name?


ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 API key expired. Please renew the API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key expired. Please renew the API key."
]

In [ ]:
# To install: pip install tavily-python
from tavily import TavilyClient
client = TavilyClient("tvly-dev-***********************")
response = client.search(
    query="What are AI startups that are based in NYC?"
)
print(response)



{'query': 'What are AI startups that are based in NYC?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.builtinnyc.com/companies/type/artificial-intelligence-companies', 'title': 'Top NYC, NY AI Companies 2025 | Built In', 'content': '[![Image 6: Coinbase](https://builtin.com/sites/www.builtin.com/files/2025-05/502x282%20(2).png)](https://www.builtinnyc.com/company/coinbase) Datadog (NASDAQ: DDOG) is a [...]nnyc.com/companies/type/big-data-analytics-companies)[NYC Biotech Companies](https://www.builtinnyc.com/companies/type/biotech-companies)[NYC Blockchain Companies](https://www.builtinnyc.com/companies/type/blockchain-companies)[NYC Business Intelligence Companies](https://www.builtinnyc.com/companies/type/business-intelligence-industry-companies)[NYC Cannabis Companies](https://www.builtinnyc.com/companies/type/cannabis-companies)[NYC Chemical Companies](https://www.builtinnyc.com/companies/type/chemical-companies)[NYC Cloud Companies](ht

In [ ]:
from tavily import TavilyClient

client = TavilyClient("tvly-dev-****************************")  # pega aquí tu key directo
response = client.search(query="What are AI startups that are based in NYC?")
print(response)


{'query': 'What are AI startups that are based in NYC?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.builtinnyc.com/companies/type/artificial-intelligence-companies', 'title': 'Top NYC, NY AI Companies 2025 | Built In', 'content': '[![Image 6: Coinbase](https://builtin.com/sites/www.builtin.com/files/2025-05/502x282%20(2).png)](https://www.builtinnyc.com/company/coinbase) Datadog (NASDAQ: DDOG) is a [...]nnyc.com/companies/type/big-data-analytics-companies)[NYC Biotech Companies](https://www.builtinnyc.com/companies/type/biotech-companies)[NYC Blockchain Companies](https://www.builtinnyc.com/companies/type/blockchain-companies)[NYC Business Intelligence Companies](https://www.builtinnyc.com/companies/type/business-intelligence-industry-companies)[NYC Cannabis Companies](https://www.builtinnyc.com/companies/type/cannabis-companies)[NYC Chemical Companies](https://www.builtinnyc.com/companies/type/chemical-companies)[NYC Cloud Companies](ht

In [140]:
import os
from dotenv import load_dotenv

# Cargar el archivo .env
load_dotenv()

# Leer la API Key
api_key = os.getenv("TAVILY_API_KEY")

print("========== DIAGNÓSTICO ==========")
if api_key is None:
    print("❌ No se encontró la variable TAVILY_API_KEY")
else:
    print("✅ Variable encontrada")
    print("Valor leído:", api_key)

# Extra: listar todas las variables cargadas
print("\n📌 Variables de entorno cargadas:")
for key, value in os.environ.items():
    if "TAVILY" in key:
        print(f"{key} = {value}")


========== DIAGNÓSTICO ==========
✅ Variable encontrada
Valor leído: tu_api_key_tavily

📌 Variables de entorno cargadas:
TAVILY_API_KEY = tu_api_key_tavily
YOUR_ACTUAL_TAVILY_API_KEY = YOUR_ACTUAL_TAVILY_API_KEY
